In [1]:

import re
import csv
import json
from pathlib import Path
import openpyxl
from openpyxl.drawing.image import Image
import os
import cv2
import pandas as pd
from glob import glob

### 从lua到表格

In [2]:

class TranslationMods:
    def __init__(self, mod_dir):
        super().__init__()
        self.mod_info = []
        self.base_dir = mod_dir
        self.lua_dirs = glob(mod_dir + "/**/*.lua", recursive=True)

        self.lua_table_begin_pattern = r'local\s+\w+\s*=\s*\{'
        self.patterns = {
            'object_type': r'object_type\s*=\s*["\']([^"\']*)["\']',
            'name': r'name\s*=\s*["\']([^"\']*)["\']',
            'set': r'set\s*=\s*["\']([^"\']*)["\']',
            'key': r'key\s*=\s*["\']([^"\']*)["\']',
            'path': r'path\s*=\s*["\']([^"\']*)["\']',
            'px': r'px\s*=\s*(\d+)',
            'py': r'py\s*=\s*(\d+)',
            'atlas': r'atlas\s*=\s*["\']([^"\']*)["\']',
            'loc_txt': r'loc_txt\s*=\s*\{[^}]*?name\s*=\s*["\']([^"\']*)["\']\s*,\s*text\s*=',
            'rarity': r'rarity\s*=\s*(\d+)',
            'cost': r'cost\s*=\s*(\d+)',
            'pos': r"pos\s*=\s*\{\s*x\s*=\s*(\d+)\s*,\s*y\s*=\s*(\d+)",
            
        }
        self.atlasDict = {}
        
    def load_atlas(self , file_path):
        pattern  = r'SMODS\.Atlas\(\s*{\s*key\s*=\s*"(.*?)",\s*path\s*=\s*"(.*?)",\s*px\s*=\s*(\d+),\s*py\s*=\s*(\d+)\s*'
        with open(file_path, 'r', encoding='utf-8') as f:
            input_str = f.read()
            matches = re.findall(pattern, input_str)
    
            for match in matches:
                key, path, px, py = match
                
                self.atlasDict[key] = {
                    "path" : path,
                    "px" : px ,
                    "py" : py
                }
        
    def _match_table(self, s, begin_idx):
        table = ""
        n = len(s)
        l = 0 
        r = 0
        while begin_idx < n:
            table += s[begin_idx]
            if s[begin_idx] == "{":
                l = l + 1
            elif s[begin_idx] == "}":
                r = r + 1
            begin_idx += 1
            if l == r :
                return table
        return table
    
    def _table_clean(self, table):
        table = re.sub(r'--.*?(?=["\n])', '', table)
        table = table.strip().replace("\t","").replace("\n","").replace("'", '"')
        table = re.sub(r',(\s*[\]\}])', r'\1', table)
        table = table.replace('n"t', "n't")
        table = table.replace('i"m', "i'm")
        table = table.replace('n\\"t', "n't")
        
        return f"[{table[1:-1]}]"

    def conver(self, lua_string):
        results = {}
        for field, pattern in self.patterns.items():
            match = re.search(pattern, lua_string, re.DOTALL)
            if match:
                # 如果有匹配到 loc_txt，需要进一步处理多行字符串
                if field == 'loc_txt':
                    name = match.group(1)
                    table = self._match_table(lua_string,match.end()+1)
                    table = self._table_clean(table)
                    
                    try:
                        json_texts = json.loads(table)
                        results[field] = {'name': name, 'text': json_texts}
                    except Exception as e:
                        print(e)
                        print(table)
                        results[field] = ""
                        # raise e

                elif field == 'pos':
                    match = re.search(pattern, lua_string)
                    if match:
                        results[field] = { "x" : int(match.group(1)) ,"y" : int(match.group(2))}
                else:
                    results[field] = match.group(1)
            else :
                results[field] = ""
        return results
    
        
        
    def prase_files(self):
        for mod_file in self.lua_dirs :
            fname = os.path.basename(mod_file)
            with open(mod_file, 'r', encoding='utf-8') as f:
                content = f.read()

            self.load_atlas(mod_file)
            lua_table_begin_match = re.finditer(self.lua_table_begin_pattern, content)

            for match in lua_table_begin_match:
                ed = match.end() - 1
                tb = self._match_table(content ,ed)
                results = self.conver(tb)
                
                if results["object_type"] == "Atlas":
                    self.atlasDict[results["key"]] = {
                        "path" : results["path"],
                        "px" : results["px"] ,
                        "py" : results["py"]
                    }
                elif results["loc_txt"] != "" :
                    results["file_name"] = mod_file
                    results["table_begin_idx"] = ed
                    results["table_len"] = len(tb)
                
                    self.mod_info.append(results)
                    
        print(f"全部完成，一共{len(self.mod_info)}条")
        
        
    def _flatten_dict(self,item):
        return {
            'file_name': str(item['file_name']),
            'object_type': item['object_type'],
            'name': item['name'],
            'set' : item['object_type'] if item['set'] =='' else item['set'],
            'key': item['key'],
            'loc_txt_name': item['loc_txt']['name'],
            'T_loc_txt_name': item['T_loc_txt_name'] if 'T_loc_txt_name' in item.keys() else '',
            'loc_txt_text': '\n'.join(item['loc_txt']['text']),
            'T_loc_txt_text': item['T_loc_txt_text'] if 'T_loc_txt_text' in item.keys() else '',
            'cost' : item['cost'],
            'rarity': item['rarity']
        }
        
    def crop_image(self ,img_path, item):
        key = item['atlas']
        x = item['pos']['x']
        y = item['pos']['y']
        
        # 大图片路径
        large_img_path =os.path.join(self.base_dir, "assets\\1x\\" , self.atlasDict[key]["path"]) 
        # print(large_img_path)
        # 小图片的尺寸
        small_width = int(self.atlasDict[key]["px"])
        small_height = int(self.atlasDict[key]["py"])
        # 读取大图
        large_img = cv2.imread(large_img_path)
        # 计算小图的位置（裁剪区域）
        start_x = x * small_width
        start_y = y * small_height
        end_x = start_x + small_width
        end_y = start_y + small_height
        
        # 裁剪出小图
        small_img = large_img[start_y:end_y, start_x:end_x]
        cv2.imwrite(img_path, small_img)
        
    def export_excel(self,excel_path="output.xlsx",with_img = False):
        pixel_to_excel_width = 0.75
        pixel_to_excel_height = 0.75

        
        flattened_data = [self._flatten_dict(item) for item in self.mod_info]
        df = pd.DataFrame(flattened_data)
        df.to_excel(excel_path, index=False, engine='xlsxwriter')
        
        if with_img:
            wb = openpyxl.load_workbook(excel_path)
            ws = wb.active
            for idx,item in zip(range(len(self.mod_info)),self.mod_info):
                if item["atlas"] in self.atlasDict and item['pos'] != '':
                    img_path =f"tmp/tempimg-{idx+1}.png"
                    self.crop_image(img_path , item)
                    img = Image(img_path)
                    
                    cell = f'L{idx+2}'
                    img.anchor  = cell
                    ws.add_image(img)  # 将图片插入到单元格H2
                    
                    # 设置列宽和行高
                    col_letter = cell[0]  # 列字母
                    row_num = int(cell[1:])  # 行号
                    
                    # 设置列宽
                    # img_width = int(self.atlasDict[item["atlas"]]["px"])
                    # img_height = int(self.atlasDict[item["atlas"]]["py"])
                    ws.column_dimensions[col_letter].width = 80
                    ws.row_dimensions[row_num].height = 100
                    
                    # os.remove(img_path)
                
            wb.save(excel_path)
            
                
        
    def import_from_excel(self, csv):
        
        pass
    
    def export_to_lua(self):
        
        pass
    

In [3]:
def filter(T):
    return T[(T['T.loc_txt.name'].notna()) & (T['T.loc_txt.name'] != '') |
                (T['T.loc_txt.text'].notna()) & (T['T.loc_txt.text'] != '')]


In [4]:
cryptidmods = TranslationMods("cryptid")
cryptidmods.prase_files()

全部完成，一共277条


In [5]:
cryptidmods.mod_info

[{'object_type': 'Back',
  'name': 'cry-Blank',
  'set': '',
  'key': 'blank',
  'path': '',
  'px': '',
  'py': '',
  'atlas': 'blank',
  'loc_txt': {'name': 'Blank Deck', 'text': ['{C:inactive,E:1}Does nothing?']},
  'rarity': '',
  'cost': '',
  'pos': {'x': 1, 'y': 0},
  'file_name': 'cryptid\\Items\\Antimatter.lua',
  'table_begin_idx': 15,
  'table_len': 235},
 {'object_type': 'Back',
  'name': 'cry-Antimatter',
  'set': '',
  'key': 'antimatter',
  'path': '',
  'px': '',
  'py': '',
  'atlas': 'antimatter',
  'loc_txt': {'name': 'Antimatter Deck',
   'text': ['Applies the {C:legendary,E:1}upsides{}',
    'of {C:attention}every{} deck']},
  'rarity': '',
  'cost': '',
  'pos': {'x': 2, 'y': 0},
  'file_name': 'cryptid\\Items\\Antimatter.lua',
  'table_begin_idx': 394,
  'table_len': 5024},
 {'object_type': 'Blind',
  'name': 'cry-oldox',
  'set': '',
  'key': 'oldox',
  'path': '',
  'px': '',
  'py': '',
  'atlas': 'nostalgia',
  'loc_txt': {'name': 'Nostalgic Ox',
   'text': [

In [6]:
st = pd.read_csv("Cryptid-汉化-Sheet1.csv")
fed_df = filter(st)

In [7]:
for index, row in fed_df.iterrows():
    result = next((i for i,d in enumerate(cryptidmods.mod_info) if d.get("key") == row["key"]), None)
    # print(result)
    # print(row["T.loc_txt.text"])
    if row["T.loc_txt.text"] != "":
        cryptidmods.mod_info[result]["T_loc_txt_text"] = row["T.loc_txt.text"]
    
    if row["T.loc_txt.name"] != "":
        cryptidmods.mod_info[result]["T_loc_txt_name"] = row["T.loc_txt.name"]

In [8]:
cryptidmods.export_excel("cryptid.xlsx" , True)

In [7]:
st = pd.read_csv("cryptid.csv",encoding='utf-8')

In [8]:
st.head()

,file_name,object_type,name,set,key,loc_txt_name,T_loc_txt_name,loc_txt_text,T_loc_txt_text,cost,rarity
0,cryptid\Items\Antimatter.lua,Back,cry-Blank,Back,blank,Blank Deck,NaN,"{C:inactive,E:1}Does nothing?",NaN,NaN,NaN
1,cryptid\Items\Antimatter.lua,Back,cry-Antimatter,Back,antimatter,Antimatter Deck,NaN,"Applies the {C:legendary,E:1}upsides{}\nof {C:...",NaN,NaN,NaN
2,cryptid\Items\Blinds.lua,Blind,cry-oldox,Blind,oldox,Nostalgic Ox,NaN,All hands start\nwith 0 Chips,NaN,NaN,NaN
3,cryptid\Items\Blinds.lua,Blind,cry-oldhouse,Blind,oldhouse,Nostalgic House,NaN,No Full Houses,NaN,NaN,NaN
4,cryptid\Items\Blinds.lua,Blind,cry-oldarm,Blind,oldarm,Nostalgic Arm,NaN,Must play 4\nor fewer cards,出牌数必须少于等于四张牌,NaN,NaN


In [9]:
def filter(T):
    return T[(T['T_loc_txt_name'].notna()) & (T['T_loc_txt_name'] != '') |
                (T['T_loc_txt_text'].notna()) & (T['T_loc_txt_text'] != '')]


In [11]:
# ft = filter(st)
ft = st 

In [12]:
descriptions = {}

In [16]:
for objecttype, group in ft.groupby('object_type'):
    
    typee = objecttype if objecttype != "Consumable" else "Other"
    descriptions[objecttype] = {}
    for row in group.itertuples():
        
        key =  objecttype[0].lower() + "_" + "cry_" + row.key
        if key not in descriptions[objecttype].keys():
            descriptions[objecttype][key] = {}
        
        descriptions[objecttype][key]["name"] = row.loc_txt_name if pd.isna(row.T_loc_txt_name)  else row.T_loc_txt_name
        texts = row.loc_txt_text if pd.isna(row.T_loc_txt_text)  else row.T_loc_txt_text
        descriptions[objecttype][key]["text"] = str(texts).split("\n")
    

In [21]:
def dict_to_lua_table(d, indent=0):
    """将 Python 字典转换为 Lua table 格式的字符串，并添加缩进和换行。"""
    indent_str = "\t" * (indent+1)
    next_indent_str = "\t" * (indent + 2)
    
    if isinstance(d, dict):
        items = []
        for key, value in d.items():
            # 递归处理键值对，key 必须是字符串，并且放在 Lua 格式的括号中
            items.append(f'{next_indent_str}{key} = {dict_to_lua_table(value, indent + 1)}')
        return "{\n" + ",\n".join(items) + "\n" + indent_str + "}"
    
    elif isinstance(d, list):
        items = [f'{next_indent_str}"{item}"' for item in d]
        return "{\n" + ",\n".join(items) + "\n" + indent_str + "}"
    
    elif isinstance(d, str):
        # 处理字符串类型，保留双引号
        return f'"{d}"'
    
    else:
        # 处理其他类型，转换为字符串格式
        return str(d)

In [22]:
# print(dict_to_lua_table(descriptions))
table =dict_to_lua_table(descriptions)

In [23]:
res = "return {\n" + "\tdescriptions="+table + "\n}"

In [24]:
print(res)

return {
	descriptions={
		Back = {
			b_cry_blank = {
				name = "Blank Deck",
				text = {
					"{C:inactive,E:1}Does nothing?"
				}
			},
			b_cry_antimatter = {
				name = "Antimatter Deck",
				text = {
					"Applies the {C:legendary,E:1}upsides{}",
					"of {C:attention}every{} deck"
				}
			},
			b_cry_encoded = {
				name = "Encoded Deck",
				text = {
					"Start with a {C:cry_code,T:j_cry_CodeJoker}Code Joker{}",
					"and a {C:cry_code,T:j_cry_copypaste}Copy/Paste{}",
					"Only {C:cry_code}Code Cards{} appear in shop"
				}
			},
			b_cry_source_deck = {
				name = "Source Deck",
				text = {
					"All cards have a {C:cry_code}Green Seal{}",
					"Cards cannot change seals"
				}
			},
			b_cry_very_fair = {
				name = "Very Fair Deck",
				text = {
					"{C:blue}-2{} hands, {C:red}-2{} discards",
					"every round",
					"{C:attention}Vouchers{} no longer",
					"appear in the shop"
				}
			},
			b_cry_equilibrium = {
				name = "Deck of Equilibrium",
				text = {
					